In [225]:
import sys
import cv2
import numpy as np

#  Log system        
class PS_Search_Color:

    def color_detection(self, frame_in):
        # Take each frame
        #frame = cv2.imread(image_name)
        src_height, src_width, src_channels = frame_in.shape
        roiX = int(src_width / 4)
        roiWidth = roiX * 2
        roiY = src_height / 4
        roiHeight = roiY * 2
        frame = frame_in[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255

        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # detect red
        lower = np.array([150, 30, 30])
        upper = np.array([190, 255, 255])
        mask1 = cv2.inRange(hsv, lower, upper)
        lower = np.array([0, 30, 30])
        upper = np.array([10, 255, 255])
        mask2 = cv2.inRange(hsv, lower, upper)
        mask = mask1 + mask2
        redVal = float(mask.sum()) / float(max_value)
        if redVal > 0.4:
            red=round(redVal,2)
        else:
            red=round(redVal,2)

        # detect yellow
        lower = np.array([5, 100, 100])
        upper = np.array([40, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        yellowVal = float(mask.sum()) / float(max_value)
        if yellowVal > 0.15:
            yellow=round(yellowVal,2)
        else:
            yellow=round(yellowVal,2)

        # detect blue
        lower = np.array([100, 60, 60])
        upper = np.array([140, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        blueVal = float(mask.sum()) / float(max_value)
        if blueVal > 0.5:
            blue=round(blueVal,2)
        else:
            blue=round(blueVal,2)

        # detect green (gray)
        lower_green = np.array([103, 86, 65])
        upper_green = np.array([145, 133, 128])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        greenVal = float(mask.sum()) / float(max_value)
        if greenVal > 0.01:
            green=round(greenVal,2)
        else:
            green=round(greenVal,2)

        # detect white
        lower = np.array([0, 0, 140])
        upper = np.array([256, 60, 256])
        mask = cv2.inRange(hsv, lower, upper)
        whiteVal = float(mask.sum()) / float(max_value)
        if whiteVal > 0.5:
            white=round(whiteVal,2)
        else:
            white=round(whiteVal,2)

        # detect black
        lower_black = np.array([110,50,50])
        upper_black= np.array([130,255,255])
        mask = cv2.inRange(hsv, lower_black, upper_black)
        blackVal = float(mask.sum()) / float(max_value)
        if blackVal > 0.15:
            black=round(blackVal,2)
        else:
            black=round(blackVal,2)

        return red, yellow, blue, green, white, black



In [257]:
threshold = 0.8594594535018965
thr = 0.1 + threshold * 0.05
thr

0.14297297267509484

In [260]:
{'red_h_lower': 14.117106059823586,
 'red_h_upper': 32.2179291418945,
 'red_s_lower': 40.86466054597105,
 'red_v_lower': 46.13325174223935,
 'threshold': 0.8594594535018965}

color_boundaries_bank = {
    'red': ([149, 20, 44],[187, 255, 255]), #daytime
    #'red': ([136, 73, 71],[186, 255, 255]), #nighttime
    'yellow': ([ 14, 40, 46],[ 32, 255, 255]),
    'blue':([ 85, 60, 60],[130, 255, 255]),
    'green':([ 37, 50, 60],[ 85, 255, 255]),
    'white':([  0,  0,200],[179,  10, 255],[ 0, 10,200],[179, 25,255]),
    'black':([  0,  0,  0],[179,  30, 160],[ 0, 30,  0],[179,255, 60])
}

# define base threshold, default 0.3
color_baseThreshold = {
    'red': 0.22, #daytime
    #'red': 0.1, #night
    'yellow': 0.14297297267509484,#daytime
    'blue':0.3,
    'green':0.3,
    'white':0.3,
    'black':0.3
}

clr_list = {'red':0,'yellow':1,'blue':2,'green':3,'white':4,'black':5}

import sys
import cv2
import numpy as np

#  Log system        
class DL_Search_Color:

    def color_detection(self, frame_in):
        
        color_to_be_detected = ['red','yellow','blue','green','white','black']
        # Take each frame
        #frame = cv2.imread(image_name)
        src_height, src_width, src_channels = frame_in.shape
        roiX = int(src_width / 4)
        roiWidth = roiX * 2
        roiY = src_height / 4
        roiHeight = roiY * 2
        frame = frame_in[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255

        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        res = [False] * len(color_to_be_detected)
        for idx,color in enumerate(color_to_be_detected):
            color_boundaries = color_boundaries_bank[color]
            assert len(color_boundaries) % 2 == 0, "Each boundary must have a lower/upper pair"

            mask = 0
            #loop over each boundary pair for this color
            for lower,upper in zip(color_boundaries[0::2], color_boundaries[1::2]):
                mask = mask + cv2.inRange(hsv, np.array(lower), np.array(upper))
                Val = float(mask.sum()) / float(max_value)
                #if Val > color_baseThreshold[color]:
                    
                res[idx] = round(Val,2)
        return res

In [261]:
search_color = DL_Search_Color()
PS_search_color = PS_Search_Color()

In [262]:
def test(color_cls):
    # import the necessary packages
    import numpy as np
    import argparse
    import cv2
    import matplotlib.pyplot as plt
    import sys, os
    import time
    #search_color = PS_Search_Color()
    
    clr_idx = clr_list[color_cls]

    color_to_be_tested = ['red','yellow','blue','green','white','black']

    target_folder = 'color_data/12/'
    this_color_count = 0
    fp = 0
    fn = 0
    ps_fp = 0
    ps_fn = 0
    #ps_count = 0
    total = 0
    for idx, color in enumerate(color_to_be_tested):
        for file in os.listdir(target_folder + color):
            if(file != os.path.basename("__file__")):
                if(file[-3:] == 'jpg'):
            
                    origin_img = cv2.imread(target_folder + color + '/' + file)
                    #print(origin_img)
                    #plt.figure()
                    #plt.imshow(origin_img)
            
                    #plt.show()
                    #print('red yellow blue green white black')
                    #print(search_color.color_detection(origin_img))
                    #print(search_color.color_detection(origin_img))
                    res = search_color.color_detection(origin_img)
                    ps_res = PS_search_color.color_detection(origin_img)
                    
                    
                    total += 1
                    
                    if res[clr_idx]>=color_baseThreshold[color] and clr_idx != idx:
                        fp = fp +1
                    elif clr_idx == idx:
                        this_color_count += 1
                        if res[clr_idx]<color_baseThreshold[color]:
                            fn = fn + 1
                    if ps_res[clr_idx]>=color_baseThreshold[color] and clr_idx != idx:
                        ps_fp = ps_fp +1
                    elif ps_res[clr_idx]<color_baseThreshold[color] and clr_idx == idx:
                        ps_fn = ps_fn + 1
    print("total",total)
    print("color :",this_color_count)
    print(fp/(total-this_color_count) + fn/this_color_count)
    print("dl  fp:",fp)
    print("dl  fn:",fn)
    print("ps  fp:",ps_fp)
    print("ps  fn:",ps_fn)
    print('\n')
    return fp,fn

color_to_be_tested = ['red','yellow','blue','green','white','black']
for color in color_to_be_tested:   
    print(color)
    fp,fn = test(color)

red


/home/max/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/max/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


('total', 2132)
('color :', 393)
0.0113282033048
('dl  fp:', 2)
('dl  fn:', 4)
('ps  fp:', 5)
('ps  fn:', 5)


yellow
('total', 2132)
('color :', 386)
0.0140988432479
('dl  fp:', 2)
('dl  fn:', 5)
('ps  fp:', 0)
('ps  fn:', 48)


blue
('total', 2132)
('color :', 305)
0.349163279406
('dl  fp:', 590)
('dl  fn:', 8)
('ps  fp:', 562)
('ps  fn:', 17)


green
('total', 2132)
('color :', 29)
0.931034482759
('dl  fp:', 0)
('dl  fn:', 27)
('ps  fp:', 59)
('ps  fn:', 29)


white
('total', 2132)
('color :', 506)
0.494776094006
('dl  fp:', 14)
('dl  fn:', 246)
('ps  fp:', 659)
('ps  fn:', 1)


black
('total', 2132)
('color :', 513)
0.860312541012
('dl  fp:', 121)
('dl  fn:', 403)
('ps  fp:', 81)
('ps  fn:', 194)




In [264]:
#  Log system        
class test_Search_Color:

    def color_detection(self, frame_in, arg1, arg2, arg3, arg4):
        
        #color_to_be_detected = ['red','yellow','blue','green','white','black']
        # Take each frame
        #frame = cv2.imread(image_name)
        src_height, src_width, src_channels = frame_in.shape
        roiX = int(src_width / 4)
        roiWidth = roiX * 2
        roiY = src_height / 4
        roiHeight = roiY * 2
        frame = frame_in[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255
        
        src_height, src_width, src_channels = frame.shape
        max_value = src_height * src_width * 255

        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        

        
        lower = np.array([arg1, arg3, arg4])
        upper = np.array([arg2, 255, 255])
        mask = cv2.inRange(hsv, lower, upper)
        Val = float(mask.sum()) / float(max_value)
    
        res = round(Val,2)


        return res

In [268]:
test_search_color = test_Search_Color()

In [267]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
import sys, os
import time
#search_color = PS_Search_Color()
def test(color_cls,arg1,arg2,arg3,arg4,arg5):

    clr_idx = clr_list[color_cls]

    color_to_be_tested = ['red','yellow','blue','green','white','black']

    target_folder = 'color_data/12/'
    this_color_count = 0
    fp = 0
    fn = 0
    #ps_count = 0
    total = 0
    for idx, color in enumerate(color_to_be_tested):
        for file in os.listdir(target_folder + color):
            if(file != os.path.basename("__file__")):
                if(file[-3:] == 'jpg'):
            
                    origin_img = cv2.imread(target_folder + color + '/' + file)
                    #print(origin_img)
                    #plt.figure()
                    #plt.imshow(origin_img)
            
                    #plt.show()
                    #print('red yellow blue green white black')
                    #print(search_color.color_detection(origin_img))
                    #print(search_color.color_detection(origin_img))
                    res = test_search_color.color_detection(origin_img,arg1,arg2,arg3,arg4)
                    #ps_res = PS_search_color.color_detection(origin_img)E
                    
                    total += 1
                    
                    if res>=arg5 and clr_idx != idx:
                        fp = fp +1
                    elif clr_idx == idx:
                        this_color_count += 1
                        if res<arg5:
                            fn = fn + 1

    return total,this_color_count,fp,fn



In [ ]:
"""
color_boundaries_bank = {
    'red': ([140, 50, 60],[190, 255, 255]),
    'yellow': ([ 10, 50, 60],[ 37, 255, 255]),
    'blue':([ 85, 60, 60],[130, 255, 255]),
    'green':([ 37, 50, 60],[ 85, 255, 255]),
    'white':([  0,  0,200],[179,  10, 255],[ 0, 10,200],[179, 25,255]),
    'black':([  0,  0,  0],[179,  30, 160],[ 0, 30,  0],[179,255, 60])
}
"""

import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pprint import pprint
from hyperopt import hp
import hyperopt.pyll.stochastic
from __future__ import division

# minimize the objective over the space
from hyperopt import fmin, tpe
import hyperopt

# define an objective function
def objective(args):
    red_h_lower,red_h_upper,red_s_lower,red_v_lower,threshold = args
    thr = 0.1 + threshold * 0.05
    print(red_h_lower,red_h_upper,red_s_lower,red_v_lower,thr)
    total,this_color_count,fp,fn = test('blue',red_h_lower,red_h_upper,red_s_lower,red_v_lower,thr)

    print(fp/(total-this_color_count) + fn/this_color_count)

   #print(a)
    return {
        'loss':fp/(total-this_color_count) + fn/this_color_count,
        'status': STATUS_OK,
        'eval_time':time.time()
    }

# define a search space
space = [hp.uniform('red_h_lower',60,100),hp.uniform('red_h_upper',115,145),
         hp.uniform('red_s_lower', 20,100),hp.uniform('red_v_lower',30,100),
         hp.uniform('threshold',0,10)]
        


#pprint(trials.trials)
print('\n')


trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=500,trials= trials)


pprint(best)
print('\n')
# -> {'a': 1, 'c2': 0.01420615366247227}
pprint(trials.losses())
#pprint(hyperopt.space_eval(space, best))
print('\n')
# -> ('case 2', 0.01420615366247227}



(86.8682235698903, 140.3708259124319, 68.71252398339662, 77.48026206689329, 0.1925424845720448)


/home/max/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.321801394385
(89.7820654412902, 142.2906671933979, 96.42416939963138, 61.5215986272904, 0.19468550031793716)
0.284897754089
(63.01558984457245, 141.5314872282608, 87.00844193351223, 87.32607165288087, 0.5273454373523063)
0.56285050293
(72.89005490196655, 122.64613792429402, 54.13075038719234, 94.02448809915902, 0.16937649251758977)
0.426376663347
(75.43663728242647, 143.44341106497197, 95.89400884464375, 74.68523063924187, 0.24521250747523335)
0.350401536156
(86.50618023924171, 119.47477119879548, 20.146911446527067, 31.52822619555582, 0.4221023675967185)
0.488774036089
(88.5207692450578, 123.94602622749413, 83.47829668943974, 60.11559726737693, 0.5428352269112645)
0.492941039238
(85.39510958798877, 116.01165458172179, 85.00645792094056, 60.83271997128656, 0.3173328816516581)
0.28157061204
(69.16597616572362, 141.5738913144396, 97.1555524478899, 77.2207612424978, 0.13042710902279323)
0.259253277343
(65.17225196667113, 120.77145998424216, 32.46593753073286, 93.84501943945453, 0.573418